## EDA and Feature engineering

The purpose of this notebook is to derive additional features from the original dataset and conduct brief EDA.
The original dataset is taken from Kaggle (Store Item Demand Forecasting Challenge)

## 00a. import required modules

In [ ]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy import stats
import numpy as np
from feature_engine.categorical_encoders import OneHotCategoricalEncoder

## 00b. user-defined function(s)

In [ ]:
# plot the histograms and Q-Q plots to have a quick look at the variable distribution

def diagnostic_plots(df, variable):
    
    # function to plot a histogram and a Q-Q plot
    # side by side, for a certain variable
    
    plt.figure(figsize=(15,6))
    plt.subplot(1, 2, 1)
    sns.distplot(df[variable], fit=norm);

    plt.subplot(1, 2, 2)
    stats.probplot(df[variable], dist="norm", plot=plt)

    plt.show()

## 01. import original data

In [ ]:
# importing data
df_train=pd.read_csv("../data/interim/train.csv")

In [ ]:
df_train.shape

In [ ]:
df_train.head()

In [ ]:
df_train.dtypes

## 02. extracting additional features from the original dataset

In [ ]:
#converting date column to date format
df_train['date']=pd.to_datetime(df_train['date'])

In [ ]:
df_train.dtypes

In [ ]:
df_train.head()

In [ ]:
#extracting weekday, week of year, year and month
df_train['weekday']=df_train['date'].dt.dayofweek
df_train['week']=df_train['date'].dt.week
df_train['year']=df_train['date'].dt.year
df_train['month']=df_train['date'].dt.month

In [ ]:
df_train.tail()

## 03. eda: target variable (sales) by year, month, day

In [ ]:
#annual sales as boxplots
px.box(x=df_train['year'],y=df_train['sales'],title="Yearly Sales")

In [ ]:
#weekday sales as boxplots
px.box(x=df_train['weekday'],y=df_train['sales'],title="Yearly Sales")

In [ ]:
#monthly sales
px.box(x=df_train['month'],y=df_train['sales'],title="Monthly Sales")

In [ ]:
#observations: sales were increasing year over year; there is a strong seasonality with peak in July and valleys in Jan and Dec
#also sales are lower on Sundays (weekday=0) but higher on Saturdays (weekday=6)

## 04. checking for outliers in the target variable (only numeric variable)

In [ ]:
outlier_cols=['sales']

In [ ]:
#what stdev# should be chosen for capping
df_capped=pd.DataFrame(outlier_cols,columns=['col'])
capped_list=[]

In [ ]:
for stdev in np.arange(1.5,3.1,0.5):
    capped_list=[]
    for col in outlier_cols:
        upper_thres=df_train[col].mean()+stdev*df_train[col].std()
        lower_thres=df_train[col].mean()-stdev*df_train[col].std()
    
        num_rec_cnt = len(df_train[df_train[col]>upper_thres])+len(df_train[df_train[col]<lower_thres])
        pctg_rec_cnt=num_rec_cnt/len(df_train)*100
        capped_list.append(pctg_rec_cnt)
    df_capped[stdev]=capped_list

In [ ]:
print('Percentage of observations to be capped depending on stdev value')
df_capped

In [ ]:
#observations: 1.5 and 2.0 stdev cannot be chosen as too many observations to be capped
#3 stdev caps too few observations
#we would pick 2.5 stdev for capping

In [ ]:
#capping outliers
thresh_dict={}
for col in outlier_cols:
    upper_thres=df_train[col].mean()+2.5*df_train[col].std()
    lower_thres=df_train[col].mean()-2.5*df_train[col].std()
    thresh_dict.update({col:{'upper':upper_thres,'lower':lower_thres}})
    df_train[col]=df_train[col].apply(lambda x: x if x>lower_thres else lower_thres)
    df_train[col]=df_train[col].apply(lambda x: x if x<upper_thres else upper_thres)

In [ ]:
#outlier capping library
pd.to_pickle(thresh_dict,'../data/processed/thres_dic.pickle')

## 05. checking for missing values and transformation

In [ ]:
#checking for missing values
df_train.isnull().sum()

In [ ]:
#observation: no missing values

In [ ]:
#checking for skewness
df_train['sales'].skew()

In [ ]:
df_train['sales'].hist()

In [ ]:
#histogram and q-q plot
diagnostic_plots(df_train, 'sales')

In [ ]:
#observation: there is some skewness towards right, so let's transform the target variable

In [ ]:
#checking for negative values
if any(df_train['sales']<0)==False:
    print("no negative values")

In [ ]:
#log transform
df_train['sales']=np.log1p(df_train['sales'])

In [ ]:
diagnostic_plots(df_train, 'sales')

In [ ]:
df_train['sales'].skew()

In [ ]:
#observation: log-tranformed target variable is closer to normal distribution 

In [ ]:
df_train.head()

## 06. encoding categorical features

In [ ]:
#list of features to be treated as categorical
cat=['store','item','weekday','week','month']

In [ ]:
#convert features to be treated as categorical to the object type
for col in cat:
    df_train[col]=df_train[col].astype('object')

In [ ]:
#one-hot encoding
ohe_enc = OneHotCategoricalEncoder(
    top_categories=None,
    variables=cat,
    drop_last=False)

In [ ]:
ohe_enc.fit(df_train[cat])

In [ ]:
encoded = ohe_enc.transform(df_train[cat])

In [ ]:
df_train=pd.concat([df_train,encoded],axis=1)

In [ ]:
df_train.head()

In [ ]:
#dropping categorical variables
df_train.drop(cat,axis=1,inplace=True)

In [ ]:
df_train.reset_index(drop=True,inplace=True)

In [ ]:
#list of columns
df_train.columns.tolist()

## 07. train/test split

In [ ]:
#we use 2017 data as a hold-out sample (test data)
df_train_train=df_train[df_train['year']!=2017]
df_train_test=df_train[df_train['year']==2017]

In [ ]:
df_train_train.shape

In [ ]:
df_train_test.shape

In [ ]:
df_train_train.head()

## 08. exporting train and test datasets

In [ ]:
df_train_train.to_csv('../data/processed/df_train_train.csv.gz',index=False,compression='gzip')
df_train_test.to_csv('../data/processed/df_train_test.csv.gz',index=False,compression='gzip')